# Medical Image Analysis Tutorial

## Outline
Use these links to jump to specific sections of this assignment!

- [1. Introduction](#1)
- [2. Installation](#2)
    - [2.1 Install mmclassification in CS GPU Farm](#2.1)
    - [2.2 2.2 Verify mmclassification and Pytorch in CS GPU Farm](#2.2)
- [3. Download and check the Dataset](#3)
    - [3.1 Download](#3.1)
    - [3.1 Refomating](#3.2)
- [4. Train a new classifier on customized dataset](#4)
- [5. Test](#5)
- [5. Improvement](#6)
    

<a name='1'></a>
## 1. Introduction¶
Among a variety of medical imaging modalities, chest X-ray is one of the most common and readily available examinations for diagnosing chest diseases. It is primarily used to screen diseases such as lung cancer, pneumonia, tuberculosis and pneumothorax to detect them at their earliest and most treatable stage. However, the problem lies in the heavy workload of reading chest X-rays. Radiologists usually read tens or hundreds of X-rays every day. Several studies regarding radiologic errors have reported that 20-30% of exams are misdiagnosed. To compensate for this shortcoming, many hospitals equip radiologists with **computer-aided diagnosis systems**. In this toturial, we will build and train **a multi-label classifier** to detect possible chest diseases among the chest X-ray images. The overall workfolw is shown as below:


![Cat](figures/tutorial_overall.jpg)

Through the tutorial, you will learn:
- How to train a chest diseases multi-label classifier?
- How to infer the trained model?



<a name='2'></a>
## 2. Installation
We use the MMClassification package to construct our training and testing pipeline. MMClassification is an open source image classification toolbox based on PyTorch.
<a name='2.1'></a>
### 2.1 Install mmclassification in CS GPU Farm

1. Create a conda virtual environment and activate it. We call the new environment as open-mmlab.
    ```shell
    conda create -n open-mmlab python=3.7 -y
    conda activate open-mmlab
    ```  
2. Install Pytorch and Torchvison following the [offcial guidance](https://pytorch.org/get-started/locally/).
    ```shell
    conda install pytorch==1.7.1 torchvision==0.8.2 torchaudio==0.7.2 cudatoolkit=10.1 -c pytorch
    ``` 
3. Install mmcv from source code.
    ```shell
    git clone https://github.com/open-mmlab/mmcv.git
    cd mmcv
    MMCV_WITH_OPS=1 pip install -e .  # package mmcv-full will be installed after this step
    cd ..
    ``` 
4. Install mmclassification form source code
    ```shell
    git clone https://github.com/open-mmlab/mmclassification.git
    cd mmclassification
    python setup.py develop
    ``` 
    
<a name='2.2'></a>    
### 2.2 Verify mmclassification and Pytorch in CS GPU Farm
```python
# check pytorch
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
# check mmclssification
import mmcls
print(mmcls.__version__)
# check mmcv
import mmcv
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(mmcv.__version__)
print(get_compiling_cuda_version())
print(get_compiler_version())
```

<a name='3'></a>
## 3. Download and check the Dataset

For this assignment, we will be using the [ChestX-ray8 dataset](https://arxiv.org/abs/1705.02315) which contains 108,948 frontal-view X-ray images of 32,717 unique patients. In summary:
- Each image in the data set contains multiple text-mined labels identifying 14 different pathological conditions. 
- We will use this data to develop a single model that will provide binary classification predictions for each of the 14 labeled pathologies. 
- In other words it will predict 'positive' or 'negative' for each of the pathologies.
 
This dataset has been annotated by consensus among four different radiologists for our 14 pathologies:
- `Cardiomegaly`, 
- `Emphysema`, 
- `Effusion`, 
- `Hernia`, 
- `Infiltration`, 
- `Mass`, 
- `Nodule`, 
- `Atelectasis`,
- `Pneumothorax`,
- `Pleural_Thickening`, 
- `Pneumonia`, 
- `Fibrosis`, 
- `Edema`, 
- `Consolidation`

<a name='3.1'></a>
### 3.1 Download
We downlaod the whole dataset via the kaggle websites:
- Register your [Kaggle] (https://www.kaggle.com/) account 
- Download the ChesyX-ray8 dataset from https://www.kaggle.com/nih-chest-xrays/data, and FTP the downloaded dataset to your GPU farm via FTP tool.
- Or you can download the dataset to GPU farm directly.
   - Typing cmd "pip install kaggle" in GPU farm to install [kaggle api](https://github.com/Kaggle/kaggle-api) so that you can download the dataset via the command line.
   - Typing cmd "kaggle datasets download -d nih-chest-xrays/data" to download dataset.
  
<a name='3.2'></a>
### 3.2 Refomating
- We rename the dataset to "cxr14" and move to the *data* folder under mmclssification.
    ```shell
    mv nih-chest-xrays cxr14
    mkdir data
    mv cxr14 data
    ``` 
- The directory should be like that
```
cxr14
     |--images
          |--00012163_001.png
          |--00012163_001.png
          ...   
     |--labels
          |--train.csv
          |--val.csv
          |--test.csv   
``` 
- We convert the csv annotation to txt format

```python
import pandas
import numpy as np
CLASSES = ["Atelectasis", "Cardiomegaly", "Effusion", "Infiltration", "Mass", "Nodule", "Pneumonia",
           "Pneumothorax", "Consolidation", "Edema", "Emphysema", "Fibrosis", "Pleural_Thickening", "Hernia"]

for item in ["train", "val", "test"]:
    object_csv = f"/apdcephfs/share_1364275/yuanfengji/data/medical/xray/cxr14/labels/{item}.csv"
    target_txt = f"/apdcephfs/share_1364275/yuanfengji/data/medical/xray/cxr14/labels/{item}.txt"

    object = pandas.read_csv(object_csv)

    image_idx = list(object["Image Index"])
    label_idx = list(object["Finding Labels"])
    write_strings = []

    for image_id, label_id in zip(image_idx, label_idx):
        num_labels = np.zeros((14))
        lables = label_id.split("|")
        for item in lables:
            if item == "No Finding":
                pass
            else:
                idx = CLASSES.index(item)
                num_labels[idx] = 1

        write_strings.append(image_id+" "+' '.join(str(int(x)) for x in num_labels))

    with open(target_txt, 'w') as f:
        for item in write_strings:
            f.write(item+'\n')
    f.close()
``` 

- Check the convert annotation
```
!cat data/cxr/labels/train.txt
``` 
The output shold be:
```
00000001_000.png 0 1 0 0 0 0 0 0 0 0 0 0 0 0
00000001_001.png 0 1 0 0 0 0 0 0 0 0 1 0 0 0
00000001_002.png 0 1 1 0 0 0 0 0 0 0 0 0 0 0
00000002_000.png 0 0 0 0 0 0 0 0 0 0 0 0 0 0
00000004_000.png 0 0 0 0 1 1 0 0 0 0 0 0 0 0
00000005_000.png 0 0 0 0 0 0 0 0 0 0 0 0 0 0
00000005_001.png 0 0 0 0 0 0 0 0 0 0 0 0 0 0
00000005_002.png 0 0 0 0 0 0 0 0 0 0 0 0 0 0
00000005_003.png 0 0 0 0 0 0 0 0 0 0 0 0 0 0
00000005_004.png 0 0 0 0 0 0 0 0 0 0 0 0 0 0
00000005_005.png 0 0 0 0 0 0 0 0 0 0 0 0 0 0
...
```

<a name='4'></a>
## 4. Train a new classifier on customized dataset

### 4.1 Support  CXR14 dataset to mmcls package.

In ```mmclassification/mmcls/datasets```, We follow the file ```imagenet.py``` to write a dataset class file "cxr14.py". Simply, we can copy 'imagenet.py' and re-write it's classes here:

```python
@DATASETS.register_module()
class CXR14(BaseDataset):

    IMG_EXTENSIONS = ('.jpg', '.jpeg', '.png', '.ppm', '.bmp', '.pgm', '.tif')
    CLASSES = ["Atelectasis", "Cardiomegaly", "Effusion", "Infiltration", "Mass", "Nodule", "Pneumonia",
               "Pneumothorax", "Consolidation", "Edema", "Emphysema", "Fibrosis", "Pleural_Thickening", "Hernia"]

    def load_annotations(self):
        if self.ann_file is None:
            folder_to_idx = find_folders(self.data_prefix)
            samples = get_samples(
                self.data_prefix,
                folder_to_idx,
                extensions=self.IMG_EXTENSIONS)
            if len(samples) == 0:
                raise (RuntimeError('Found 0 files in subfolders of: '
                                    f'{self.data_prefix}. '
                                    'Supported extensions are: '
                                    f'{",".join(self.IMG_EXTENSIONS)}'))

            self.folder_to_idx = folder_to_idx
        elif isinstance(self.ann_file, str):
            samples = []
            with open(self.ann_file) as f:
                for line in f.readlines():
                    items = line.strip().split(' ')
                    filename = items[0]
                    label = [int(i) for i in items[1:]]
                    samples.append([filename, label])
        else:
            raise TypeError('ann_file must be a str or None')
        self.samples = samples

        data_infos = []
        for filename, gt_label in self.samples:
            info = {'img_prefix': self.data_prefix}
            info['img_info'] = {'filename': filename}
            info['gt_label'] = np.array(gt_label, dtype=np.float)
            data_infos.append(info)
        return data_infos
```
It should be note that we need to import 'CXR14' class in the ```_init_.py``` under ```mmclassification/mmcls/datasets```, so that the mmcls can locate and register the CXR14 scucessfully.

### 4.2 Set the logs file for Model Training
Now we'll move on to model training, Suppose we are at 'mmclassification/configs/__base__' directory.
#### 4.2.1 Set the datset config
In ```_base_/datasets```, we add ```cxr14_bs32.py``` to describle the basic config about data. Note that we need to declare the necessary data paths and data pipelines in it like this:
```python
dataset_type = 'CXR14'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='RandomResizedCrop', size=224),
    dict(type='RandomFlip', flip_prob=0.5, direction='horizontal'),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='ImageToTensor', keys=['img']),
    dict(type='ToTensor', keys=['gt_label']),
    dict(type='Collect', keys=['img', 'gt_label'])
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', size=(256, 256)),
    dict(type='CenterCrop', crop_size=224),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='ImageToTensor', keys=['img']),
    dict(type='Collect', keys=['img'])
]
data = dict(
    samples_per_gpu=32,
    workers_per_gpu=4,
    train=dict(
        type=dataset_type,
        data_prefix='data/medical/cxr14/images',
        ann_file='data/medical/cxr14/labels/trainval.txt',
        pipeline=train_pipeline),
    val=dict(
        type=dataset_type,
        data_prefix='data/medical/cxr14/images',
        ann_file='data/medical/cxr14/labels/test.txt',
        pipeline=test_pipeline),
    test=dict(
        type=dataset_type,
        data_prefix='data/medical/cxr14/images',
        ann_file='data/medical/cxr14/labels/val.txt',
        pipeline=test_pipeline))
evaluation = dict(
    interval=1, metric=['mAP', 'AUC', 'CP', 'OP', 'CR', 'OR', 'CF1', 'OF1'], tta=False)
```
#### 4.2.2 Set the model config
In ``_base_/models``, we added ``resnet50_cxr14.py`` to describe the basic configuration about the model. Note that we use the ``resnet-50`` model as backbone, and then add two layers on top of it:
1. A `GlobalAveragePooling2D` layer to get the average of the last convolution layers from Resnet50.
2. A `MultiLabelClsHead` layer with `sigmoid` activation to get the prediction logits for each of our classes.
Here the ''num_classes'' is set to 14 because the total number of classes for chest diseases is 14.

```python
model = dict(
    type='ImageClassifier',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(3,),
        style='pytorch'),
    neck=dict(type='GlobalAveragePooling'),
    head=dict(
        type='MultiLabelClsHead',
        in_channels=2048,
        num_classes=14,
        loss=dict(type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0))
 ```   
#### 4.2.3 Set the training schedule config
In ``_base_/schedules``, we added ``cxr14_bs32_ep20.py`` to develop the basic configuration about the training plan.Here we adopt "AdamW" optimizer and set lr to 3e-4 with a setp learning scheduler, which decay the lr decay in 10th and 15th. The total training epoch is set to 20. Meanwhile, we load weight pretrained on ImageNet datset.
```python
optimizer = dict(type='AdamW', lr=0.003, weight_decay=0.3) 
optimizer_config = dict(grad_clip=None)
lr_config = dict(policy='step', step=[10, 15])
runner = dict(type='EpochBasedRunner', max_epochs=20)
```
#### 4.2.4 Set the logging and saving config
In ``_base_/schedules``, we set the ``default_runtime.py`` as follow:
```python
# checkpoint saving
checkpoint_config = dict(interval=1)
# yapf:disable
log_config = dict(
    interval=100,
    hooks=[
        dict(type='TextLoggerHook'),
        dict(type='TensorboardLoggerHook')
    ])
# yapf:enable

dist_params = dict(backend='nccl')
log_level = 'INFO'
load_from = 'weights/resnet50_batch256_imagenet_20200708-cfb998bf.pth'
resume_from = None
workflow = [('train', 1)]
```

#### 4.2.5 Import configs for model training
we create a config file in ``mmclassification/configs/resnet/resnet50_cxr14_bs32.py``. we import these config.py files into this file as
```python
_base_ = [
    '../_base_/datasets/cxr14_bs32.py', '../_base_/models/resnet50_cxr14.py',
    '../_base_/schedules/cxr14_bs32_ep20.py', '../_base_/default_runtime.py'
]
```

### 4.3 Training
In your 'mmclassification' directory, we run cmd:
```shell
python tools/train.py configs/resnet/resnet50_cxr14_bs32.py --work_dir 'work_dirs/resnet50_cxr14_bs32'
```
It should be note that we need to state the  training configuration 'configs/resnet/resnet50_cxr14_bs32.py' and the work_dir is used for saving the training results (model and log files). In such case, the trained model and logs are saved in work_dirs/resnet50_cxr14_bs32. Now, we change the directory to work_dirs/resnet50_cxr14_bs32 and we should get the saved model and log files.

We show an example of training log, with a mean auc of 81.5%.
```shell
2021-10-04 16:43:02,050 - mmcls - INFO - Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.8.5 (default, Sep  4 2020, 07:30:14) [GCC 7.3.0]
CUDA available: True
GPU 0: Tesla V100-SXM2-32GB
CUDA_HOME: /usr/local/cuda
NVCC: Build cuda_11.1.TC455_06.29190527_0
GCC: gcc (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0
PyTorch: 1.8.0a0+1606899
PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2019.0.4 Product Build 20190411 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v1.6.0 (Git Hash N/A)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.1
  - NVCC architecture flags: -gencode;arch=compute_52,code=sm_52;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_61,code=sm_61;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80,code=sm_80;-gencode;arch=compute_86,code=sm_86;-gencode;arch=compute_86,code=compute_86
  - CuDNN 8.0.5
  - Magma 2.5.2
  - Build settings: BLAS=MKL, BUILD_TYPE=Release, CXX_FLAGS= -fvisibility-inlines-hidden -DUSE_PTHREADPOOL -fopenmp -DNDEBUG -DUSE_FBGEMM -DUSE_QNNPACK -DUSE_PYTORCH_QNNPACK -DUSE_XNNPACK -DUSE_VULKAN_WRAPPER -O2 -fPIC -Wno-narrowing -Wall -Wextra -Werror=return-type -Wno-missing-field-initializers -Wno-type-limits -Wno-array-bounds -Wno-unknown-pragmas -Wno-sign-compare -Wno-unused-parameter -Wno-unused-variable -Wno-unused-function -Wno-unused-result -Wno-unused-local-typedefs -Wno-strict-overflow -Wno-strict-aliasing -Wno-error=deprecated-declarations -Wno-stringop-overflow -Wno-psabi -Wno-error=pedantic -Wno-error=redundant-decls -Wno-error=old-style-cast -fdiagnostics-color=always -faligned-new -Wno-unused-but-set-variable -Wno-maybe-uninitialized -fno-math-errno -fno-trapping-math -Werror=format -Werror=cast-function-type -Wno-stringop-overflow, FORCE_FALLBACK_CUDA_MPI=1, PERF_WITH_AVX=1, PERF_WITH_AVX2=1, PERF_WITH_AVX512=1, USE_CUDA=ON, USE_EXCEPTION_PTR=1, USE_GFLAGS=OFF, USE_GLOG=OFF, USE_MKL=ON, USE_MKLDNN=ON, USE_MPI=ON, USE_NCCL=ON, USE_NNPACK=ON, USE_OPENMP=ON, 

TorchVision: 0.9.0a0
OpenCV: 4.4.0
MMCV: 1.3.4
MMCV Compiler: n/a
MMCV CUDA Compiler: n/a
MMClassification: 0.11.0+
------------------------------------------------------------

2021-10-04 16:43:02,050 - mmcls - INFO - Distributed training: False
2021-10-04 16:43:02,185 - mmcls - INFO - Config:
model = dict(
    type='ImageClassifier',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(3, ),
        style='pytorch'),
    neck=dict(type='GlobalAveragePooling'),
    head=dict(
        type='MultiLabelClsHead',
        in_channels=2048,
        num_classes=14,
        loss=dict(type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0)))
dataset_type = 'CXR14'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='RandomResizedCrop', size=224),
    dict(type='RandomFlip', flip_prob=0.5, direction='horizontal'),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='ImageToTensor', keys=['img']),
    dict(type='ToTensor', keys=['gt_label']),
    dict(type='Collect', keys=['img', 'gt_label'])
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', size=(256, 256)),
    dict(type='CenterCrop', crop_size=224),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='ImageToTensor', keys=['img']),
    dict(type='Collect', keys=['img'])
]
data = dict(
    samples_per_gpu=32,
    workers_per_gpu=4,
    train=dict(
        type='CXR14',
        data_prefix='data/medical/cxr14/images',
        ann_file='data/medical/cxr14/labels/trainval.txt',
        pipeline=[
            dict(type='LoadImageFromFile'),
            dict(type='RandomResizedCrop', size=224),
            dict(type='RandomFlip', flip_prob=0.5, direction='horizontal'),
            dict(
                type='Normalize',
                mean=[123.675, 116.28, 103.53],
                std=[58.395, 57.12, 57.375],
                to_rgb=True),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='ToTensor', keys=['gt_label']),
            dict(type='Collect', keys=['img', 'gt_label'])
        ]),
    val=dict(
        type='CXR14',
        data_prefix='data/medical/cxr14/images',
        ann_file='data/medical/cxr14/labels/test.txt',
        pipeline=[
            dict(type='LoadImageFromFile'),
            dict(type='Resize', size=(256, 256)),
            dict(type='CenterCrop', crop_size=224),
            dict(
                type='Normalize',
                mean=[123.675, 116.28, 103.53],
                std=[58.395, 57.12, 57.375],
                to_rgb=True),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img'])
        ]),
    test=dict(
        type='CXR14',
        data_prefix='data/medical/cxr14/images',
        ann_file='data/medical/cxr14/labels/val.txt',
        pipeline=[
            dict(type='LoadImageFromFile'),
            dict(type='Resize', size=(256, 256)),
            dict(type='CenterCrop', crop_size=224),
            dict(
                type='Normalize',
                mean=[123.675, 116.28, 103.53],
                std=[58.395, 57.12, 57.375],
                to_rgb=True),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img'])
        ]))
evaluation = dict(
    interval=1,
    metric=['mAP', 'AUC', 'CP', 'OP', 'CR', 'OR', 'CF1', 'OF1'],
    tta=False)
optimizer = dict(
    type='Adam', lr=0.0001, betas=(0.9, 0.999), eps=1e-08, weight_decay=1e-05)
optimizer_config = dict(grad_clip=None)
lr_config = dict(policy='step', step=[15, 18])
runner = dict(type='EpochBasedRunner', max_epochs=20)
checkpoint_config = dict(interval=1)
log_config = dict(
    interval=100,
    hooks=[dict(type='TextLoggerHook'),
           dict(type='TensorboardLoggerHook')])
dist_params = dict(backend='nccl')
log_level = 'INFO'
load_from = 'weights/resnet50_batch256_imagenet_20200708-cfb998bf.pth'
resume_from = None
workflow = [('train', 1)]
work_dir = './work_dirs/res50_b128_cxr14_ep20'
gpu_ids = range(0, 1)

2021-10-04 16:43:06,193 - mmcls - INFO - load checkpoint from weights/resnet50_batch256_imagenet_20200708-cfb998bf.pth
2021-10-04 16:43:06,194 - mmcls - INFO - Use load_from_local loader
2021-10-04 16:43:06,479 - mmcls - WARNING - The model and loaded state dict do not match exactly

size mismatch for head.fc.weight: copying a param with shape torch.Size([1000, 2048]) from checkpoint, the shape in current model is torch.Size([14, 2048]).
size mismatch for head.fc.bias: copying a param with shape torch.Size([1000]) from checkpoint, the shape in current model is torch.Size([14]).
2021-10-04 16:43:06,484 - mmcls - INFO - Start running, host: root@ts-e961c55f4d024bdabd98f3f81128cc20-launcher, work_dir: /apdcephfs/share_1364275/yuanfengji/project/mmclassification/work_dirs/res50_b128_cxr14_ep20
2021-10-04 16:43:06,485 - mmcls - INFO - workflow: [('train', 1)], max: 20 epochs
2021-10-04 16:44:23,656 - mmcls - INFO - Epoch [1][100/676]	lr: 1.000e-04, eta: 2:52:11, time: 0.770, data_time: 0.376, memory: 10963, loss: 2.8662
2021-10-04 16:45:40,562 - mmcls - INFO - Epoch [1][200/676]	lr: 1.000e-04, eta: 2:50:49, time: 0.769, data_time: 0.395, memory: 10963, loss: 2.0961
2021-10-04 16:46:52,274 - mmcls - INFO - Epoch [1][300/676]	lr: 1.000e-04, eta: 2:45:41, time: 0.717, data_time: 0.343, memory: 10963, loss: 2.0498
2021-10-04 16:48:07,000 - mmcls - INFO - Epoch [1][400/676]	lr: 1.000e-04, eta: 2:44:10, time: 0.747, data_time: 0.373, memory: 10963, loss: 2.0500
2021-10-04 16:49:26,798 - mmcls - INFO - Epoch [1][500/676]	lr: 1.000e-04, eta: 2:44:58, time: 0.798, data_time: 0.424, memory: 10963, loss: 1.9722
2021-10-04 16:50:56,585 - mmcls - INFO - Epoch [1][600/676]	lr: 1.000e-04, eta: 2:48:38, time: 0.898, data_time: 0.523, memory: 10963, loss: 2.0262
2021-10-04 16:54:45,665 - mmcls - INFO - Saving checkpoint at 1 epochs
2021-10-04 16:54:46,312 - mmcls - INFO - Epoch(val) [1][676]	AP-Atelectasis: 30.2678, AP-Cardiomegaly: 24.7547, AP-Effusion: 47.3578, AP-Infiltration: 37.4045, AP-Mass: 23.1604, AP-Nodule: 19.5599, AP-Pneumonia: 4.2387, AP-Pneumothorax: 36.9074, AP-Consolidation: 13.7337, AP-Edema: 12.6109, AP-Emphysema: 30.0434, AP-Fibrosis: 6.2853, AP-Pleural_Thickening: 10.3355, AP-Hernia: 1.5504, AP: 21.3007, AUC-Atelectasis: 74.3349, AUC-Cardiomegaly: 81.2567, AUC-Effusion: 80.7166, AUC-Infiltration: 67.6402, AUC-Mass: 74.6935, AUC-Nodule: 72.7235, AUC-Pneumonia: 67.9964, AUC-Pneumothorax: 83.0441, AUC-Consolidation: 71.9170, AUC-Edema: 81.8726, AUC-Emphysema: 84.3682, AUC-Fibrosis: 76.6358, AUC-Pleural_Thickening: 71.1178, AUC-Hernia: 76.1133, AUC: 76.0308, CP: 28.3498, CR: 3.2353, CF1: 5.8079, OP: 53.0561, OR: 6.9874, OF1: 12.3486
...
```

<a name='5'></a>
## 5. Test
We select ``data/cxr14/images/00030805_000.png`` as test image, which as shown blow:
![Cat](figures/00030805_000.png)

We can  get prediciton results from trained  model by running:
```python
python demo/image_demo.py \
  --img 'data/cxr14/images/00030805_000.png'\ 
  --config 'configs/resnet/resnet50_cxr14_bs32.py' \
  --checkpoint 'output/resnet50_cxr14_bs32/epoch_13.pth'
```
The expected output should be close to:
{'pred_label': [0, 1], 'pred_score': [0.9812, 0.8745], 'pred_class': 'Cardiomegaly', 'Emphysema'}


<a name='6'></a>
## 6. Improvement
- Now we can access a new covid 19 xray dataset, how can we use it to extend model to detect covid 19 ?